### Root

In [ ]:
#текущая папка
base = 'drive/My Drive/GPB'

!pip install transformers
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import importlib
import os
import pickle
import re

import seaborn as sns
from matplotlib import style
style.use('ggplot')
%config InlineBackend.figure_format = 'retina'

import sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    sys.path.append(base) 

!sudo pip3 install rutimeparser
import preprocessing
import ensemble

#чтобы не менять пути к файлам в зависимости от того, в колабе ноутбук или нет
def get_drive_path(path):
    in_colab = 'google.colab' in sys.modules
    return '/content/'+base+'/'+re.sub(r'\\','/',path) if in_colab else path

     |████████████████████████████████| 2.0MB 8.9MB/s 
     |████████████████████████████████| 890kB 50.3MB/s 
     |████████████████████████████████| 3.2MB 53.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=7905090bdd84507776065e857724bdd45a2955c6603630c8010629c8335f4fa0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#кодировки классов (каждому классу соответсвует число)
code_names_path = get_drive_path('data/Классы_кодировка.csv')
code_to_class, class_to_code = preprocessing.get_code_names(code_names_path)

In [ ]:
dataset = pd.read_excel(get_drive_path('data/dataset_29_03.xlsx'))
dataset = dataset.rename({'Класс':'class'}, axis=1)
dataset['class'] = [class_to_code[el.lower()] for el in dataset['class']]

unique, counts = np.unique(dataset['class'].values, return_counts=True)
unrepresented_classes = [class_ for class_,count in zip(unique, counts) if count < 20]
dataset['class'] = [el if el not in unrepresented_classes else -1 for el in dataset['class']]

train_data, test_data = train_test_split(dataset, test_size=0.1, stratify=dataset['class'])
train_data.to_csv(get_drive_path('data\\train_data_19900_03-30.txt'))
test_data.to_csv(get_drive_path('data\\test_data_19900_03-30.txt'))

In [ ]:
#загрузка датасета: в итоге датафреймы train_data и test_data должны иметь столбцы purp (текстовое описание транзакции), class, name_to, name_from

def load_data(path):
    data = pd.read_csv(path)
    data = data[data[target_name] != class_to_code['_неоднозначность']]
    return data

target_name = 'class'

train_path = get_drive_path('data\\train_data_19900_03-30.txt')
test_path = get_drive_path('data\\test_data_19900_03-30.txt')

train_data = load_data(train_path)
test_data = load_data(test_path) 
all_data = pd.concat((train_data, test_data))

train_data.shape, test_data.shape

((17981, 15), (1998, 15))

### Conv1d

In [ ]:
import conv
import embeddings as eb
full_corpus = pd.read_excel(get_drive_path('data\\full_text_corpus.xlsx'))['purp']
cleaned_corpus_version = 5 #cleaned_corpus_versions.txt, зависящие переменные - path_to_shortenings_file, fix_spelling
cleaned_corpus_path = get_drive_path(f'data\\cleaned_corpus_{cleaned_corpus_version}.csv')

path_to_shortenings_file = get_drive_path('data\\Сокращения.xlsx')
fix_spelling = False
stop_words = stopwords.words('russian')

cleaned_corpus = eb.get_cleaned_corpus(cleaned_corpus_path, 
                                       full_corpus, 
                                       stop_words,
                                       path_to_shortenings_file=path_to_shortenings_file,
                                       fix_spelling=fix_spelling)

conv_processor = conv.ConvPreprocessor(use_other_features=False, 
                                   path_to_shortenings_file=path_to_shortenings_file,
                                   fix_spelling=fix_spelling)

X_train, y_train, ids_train = conv_processor.get_data(train_data, all_data, train=True)
X_train, X_val, y_train, y_val, ids_train, ids_val = train_test_split(X_train, y_train, ids_train, test_size=0.2, stratify=y_train, random_state=42)

vocab_size = conv_processor.tokenizer.num_words if conv_processor.tokenizer.num_words < len(conv_processor.tokenizer.word_index)+1 else len(conv_processor.tokenizer.word_index)+1
padding_len = X_train.shape[1]
output_number = y_train.shape[1]

checkpoint_path = get_drive_path("saved models\\saved-model-conv.hdf5")

number_of_dimensions = 100

embedding_path = get_drive_path(f"data\\fasttext\\ft_corpus_cleaned_{number_of_dimensions}_{cleaned_corpus_version}.vec")
embedding_matrix = eb.load_fast_text_trained(embedding_path, conv_processor.tokenizer, number_of_dimensions, cleaned_corpus)

In [ ]:
conv1d_model,history = conv.train_conv1d_model(X_train, X_val, y_train, y_val,
                                               vocab_size, number_of_dimensions, embedding_matrix, padding_len, output_number, checkpoint_path)

Epoch 1/50
113/113 [==============================] - 35s 19ms/step - loss: 176.0842 - acc: 0.6108 - f1_micro: 0.6108 - f1_weighted: 0.5566 - val_loss: 19.0704 - val_acc: 0.7645 - val_f1_micro: 0.7623 - val_f1_weighted: 0.7353
Epoch 2/50
113/113 [==============================] - 2s 16ms/step - loss: 12.5702 - acc: 0.7793 - f1_micro: 0.7793 - f1_weighted: 0.7542 - val_loss: 4.0769 - val_acc: 0.8176 - val_f1_micro: 0.8161 - val_f1_weighted: 0.7942
Epoch 3/50
113/113 [==============================] - 2s 16ms/step - loss: 3.4940 - acc: 0.8302 - f1_micro: 0.8302 - f1_weighted: 0.8114 - val_loss: 2.6976 - val_acc: 0.8502 - val_f1_micro: 0.8476 - val_f1_weighted: 0.8319
Epoch 4/50
113/113 [==============================] - 2s 16ms/step - loss: 2.5715 - acc: 0.8610 - f1_micro: 0.8610 - f1_weighted: 0.8451 - val_loss: 2.4921 - val_acc: 0.8704 - val_f1_micro: 0.8697 - val_f1_weighted: 0.8598
Epoch 5/50
113/113 [==============================] - 2s 16ms/step - loss: 2.3933 - acc: 0.8829 - f1_mi

In [ ]:
#вывод метрики на тестовой выборке, определение переменных для работы ансамбля
X_test, y_test, ids_test = conv_processor.get_data(test_data, train=False)
conv1d_model.load_weights(checkpoint_path)
pred_conv_binary = conv.round_prediction(conv1d_model.predict(X_test))

print('f1 micro', f1_score(y_test, pred_conv_binary, average='micro'))
print('f1 weighted', f1_score(y_test, pred_conv_binary, average='weighted'))

pred_dict, pred_binary_dict, pred_val_dict, precision_dict = {}, {}, {}, {}
model_name = 'conv'
pred_dict[model_name] = conv_processor.y_encoder.inverse_transform(pred_conv_binary)
pred_val_dict[model_name] = conv_processor.y_encoder.inverse_transform(np.round(conv1d_model.predict(X_val)))
precision_dict[model_name] = precision_score(train_data.loc[ids_val][target_name], pred_val_dict[model_name], average=None)

f1 micro 0.8913913913913913
f1 weighted 0.8887562638430487


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Conv1d Ensemble

In [ ]:
conv_ensemble = ensemble.ConvEnsemble(full_corpus, cleaned_corpus_version, cleaned_corpus_path, path_to_shortenings_file, fix_spelling, stop_words, number_of_dimensions, embedding_path, checkpoint_path,
                                      n_splits=5,
                                      model_folder_path=get_drive_path('saved models/conv ensemble'), 
                                      model_name='conv ensemble')
train_data_ens, val_data_ens = train_test_split(train_data, test_size=0.1, stratify=train_data['class'])
conv_ensemble.train(train_data_ens, retrain=True)
conv_ens_pred = conv_ensemble.predict(test_data, mode='mode')

print('f1 micro', f1_score(test_data['class'], conv_ens_pred, average='micro'))
print('f1 weighted', f1_score(test_data['class'], conv_ens_pred, average='weighted'))

Обучение модели 1
Обучение модели 2
Обучение модели 3
Обучение модели 4
Обучение модели 5
f1 micro 0.9135357694287185
f1 weighted 0.9112634931198368


In [ ]:
#запустить эту ячейку если метрика ансамбля лучше метрики одной модели (для того, чтобы использовать его в ансамбле разных моделей)

model_name = 'conv'
pred_dict[model_name] = conv_ens_pred
pred_val_dict[model_name] = conv_ensemble.predict(val_data_ens, mode='mode')
precision_dict[model_name] = precision_score(val_data_ens['class'], pred_val_dict[model_name], average=None)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVM

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV 

In [ ]:
vectorizer = TfidfVectorizer()

X_train, X_val, y_train, y_val = train_test_split(train_data['purp'], train_data[target_name], test_size=0.2, stratify=train_data[target_name], random_state=42)

train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(test_data['purp'])

classifier_svm = LinearSVC()
classifier_svm.fit(train_vectors, y_train)
pred_svm = classifier_svm.predict(test_vectors)
#pred_svm_train = classifier_svm.predict(train_vectors)

print('f1 micro:', f1_score(test_data[target_name], pred_svm, average='micro'))
print('f1 weighted:', f1_score(test_data[target_name], pred_svm, average='weighted'))

model_name = 'svm'
pred_dict[model_name] = pred_svm
pred_val_dict[model_name] = classifier_svm.predict(vectorizer.transform(X_val))
precision_dict[model_name] = precision_score(y_val, pred_val_dict[model_name], average=None)

f1 micro: 0.91991991991992
f1 weighted: 0.9178586218373609


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Bert

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'

In [ ]:
import torch
import bert
#инициализация модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = len(all_data[target_name].unique())

bert_processor = bert.BertProccessor()

X, other_features, y = bert_processor.get_data(train_data, all_data=all_data)
X_train, X_val, y_train, y_val, other_features_train, other_features_val = train_test_split(X, y, other_features, test_size=0.2, stratify=y, random_state=42)

bert_model = bert.BertForClassification(num_classes).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(bert_model.parameters(), lr=1e-5)
epochs = 6

In [ ]:
bert_model.train_with_chunks(X_train, other_features_train, y_train,
                             X_val, other_features_val, y_val,
                             epochs=epochs, optimizer=optimizer, loss_fn=loss_fn, device=device)


Epoch: 1, train_loss: 1.8262405789087426, val_loss: 0.6689043002232146
f1:  0.8702672605790646



Epoch: 2, train_loss: 0.712943060481061, val_loss: 0.39912748400677817
f1:  0.9217706013363028



Epoch: 3, train_loss: 0.45849907815609714, val_loss: 0.35138844544345926
f1:  0.9278953229398663



Epoch: 4, train_loss: 0.31864014805685126, val_loss: 0.3692707084575647
f1:  0.9270601336302896



Epoch: 5, train_loss: 0.24492204354276198, val_loss: 0.30223000600710015
f1:  0.9359688195991092



Epoch: 6, train_loss: 0.188782346822338, val_loss: 0.33031029364235187
f1:  0.9351336302895323



In [ ]:
bert_model_path = get_drive_path('saved models/bert_model.pt')
try:
    torch.save(bert_model.state_dict(), bert_model_path)
    
except:
    bert_model = BertForClassification(num_classes)
    bert_model.load_state_dict(torch.load(bert_model_path, map_location=device))

In [ ]:
#вывод метрики на тестовой выборке, определение переменных для работы ансамбля
predicted_proba = bert_model.predict_with_chunks(X_val, other_features_val, y_val, device=device)
pred_bert_val_encoded = np.argmax(predicted_proba, axis=1)

X_test, other_features_test, y_test = bert_processor.get_data(test_data, train=False)

predicted_proba = bert_model.predict_with_chunks(X_test, other_features_test, y_test, device=device)
pred_bert_encoded = np.argmax(predicted_proba, axis=1)
pred_bert = bert_processor.le_target.inverse_transform(pred_bert_encoded)

print('f1 micro', f1_score(y_test, pred_bert_encoded, average='micro'))
print('f1 weighted', f1_score(y_test, pred_bert_encoded, average='weighted'))

model_name = 'bert'
pred_dict[model_name] = pred_bert
pred_val_dict[model_name] = bert_processor.le_target.inverse_transform(pred_bert_val_encoded)
precision_dict[model_name] = precision_score(y_val, pred_bert_val_encoded, average=None)

f1 micro 0.9354354354354354
f1 weighted 0.9342842421608064


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Bert Ensemble

In [22]:
bert_ensemble = ensemble.BertEnsemble(n_splits=5,
                           model_folder_path=get_drive_path('saved models/bert ensemble'), 
                           model_name='bert ensemble',
                           num_classes=num_classes,
                           all_data=all_data,
                           batch_size=9)

train_data_ens, val_data_ens = train_test_split(train_data, test_size=0.2, stratify=train_data['class'], random_state=42)
bert_ensemble.train(train_data_ens, retrain=True)
bert_ens_pred = bert_ensemble.predict(test_data, mode='mode')

print('f1 micro', f1_score(test_data['class'], bert_ens_pred, average='micro'))
print('f1 weighted', f1_score(test_data['class'], bert_ens_pred, average='weighted'))

Обучение модели 1


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Обучение модели 2


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Обучение модели 3


...

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Обучение модели 5


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

f1 micro 0.9419419419419419
f1 weighted 0.9400179816506825


In [ ]:
#запустить эту ячейку если метрика ансамбля лучше метрики одной модели (для того, чтобы использовать его в ансамбле разных моделей)
model_name = 'bert'
pred_dict[model_name] = bert_ens_pred
pred_val_dict[model_name] = bert_ensemble.predict(val_data_ens, mode='mode')
precision_dict[model_name] = precision_score(val_data_ens['class'], pred_val_dict[model_name], average=None)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [ ]:
#сохранение/загрузка предсказаний модели
bert_ensemble_pred_path = get_drive_path('saved models/bert_ensemble_pred.npz')
model_name = 'bert'
try:
    np.savez(bert_ensemble_pred_path, pred_dict[model_name], pred_val_dict[model_name], precision_dict[model_name])
except KeyError:
    bert_ensemble_pred_dict = np.load(bert_ensemble_pred_path)
    pred_bert, pred_bert_val, precision_bert = bert_ensemble_pred_dict['arr_0'], bert_ensemble_pred_dict['arr_1'], bert_ensemble_pred_dict['arr_2']
    
    pred_dict[model_name] = pred_bert
    pred_val_dict[model_name] = pred_bert_val
    precision_dict[model_name] = precision_bert

In [ ]:
#обучение на всем датасете (финальная версия модели)
import ensemble
importlib.reload(ensemble)

bert_ensemble = ensemble.BertEnsemble(n_splits=5,
                           model_folder_path=get_drive_path('saved models/bert ensemble full dataset'), 
                           model_name='bert ensemble',
                           num_classes=num_classes,
                           all_data=all_data)

train_data_ens = all_data
bert_ensemble.train(train_data_ens, retrain=False)

Попытка загрузить обученные модели и аттрибуты
Загружено


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import random_forest

#инициализация модели
rf_processor = random_forest.RandomForestProcessor()
X, y = rf_processor.get_data(train_data, all_data)
X_train, X_val, y_train, y_val = train_test_split(X, y, 
                                                  test_size=0.2, random_state=42)

In [ ]:
params = {'bootstrap': [False],
          'max_depth': range(40,150,20),
          'max_features': ['auto'],
          'min_samples_leaf': range(2,13,5),
          'min_samples_split': range(5,46,20),
          'n_estimators': [700]}

clf = RandomForestClassifier()
model_grid = GridSearchCV(clf,
                          params,
                          cv=5, n_jobs=-1, verbose=1, iid=False)

model_grid.fit(X, y)
model_grid.best_params_

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 16.3min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 69.7min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed: 95.2min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'bootstrap': False,
 'max_depth': 100,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 700}

In [ ]:
best_params = {'bootstrap': False,
               'max_depth': 100,
               'max_features': 'auto',
               'min_samples_leaf': 2,
               'min_samples_split': 5,
               'n_estimators': 700}

In [ ]:
clf = RandomForestClassifier(**best_params)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=100, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=700,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
#вывод метрики на тестовой выборке, определение переменных для работы ансамбля
X_test, y_test = rf_processor.get_data(test_data, train=False)
pred_rf = clf.predict(X_test)

print('f1 micro', f1_score(y_test, pred_rf, average='micro'))
print('f1 weighted', f1_score(y_test, pred_rf, average='weighted'))

model_name = 'rf'
pred_dict[model_name] = pred_rf
pred_val_dict[model_name] = clf.predict(X_val)
precision_dict[model_name] = precision_score(y_val, pred_val_dict[model_name], average=None)

f1 micro 0.8920863309352518
f1 weighted 0.8844585525124036


In [ ]:
#rf_model = pickle.load(open(get_drive_path('saved models/rf_model_0.9039.h5'), 'rb'))

#### Random Forest Ensemble

In [ ]:
import ensemble
importlib.reload(ensemble)

train_data_ens, val_data_ens = train_test_split(train_data, test_size=0.1, stratify=train_data['class'])
rf_ensemble = ensemble.RFEnsemble(n_splits=5,
                         model_folder_path=get_drive_path('saved models/rf ensemble'), 
                         model_name='rf ensemble',
                         params=params,
                         all_data=all_data)
rf_ensemble.train(train_data_ens, retrain=True)
rf_ens_pred = rf_ensemble.predict(test_data, mode='mode')

print('f1 micro', f1_score(test_data['class'], rf_ens_pred, average='micro'))
print('f1 weighted', f1_score(test_data['class'], rf_ens_pred, average='weighted'))

Обучение модели 1
Обучение модели 2
Обучение модели 3
Обучение модели 4
Обучение модели 5
f1 micro 0.8843782117163411
f1 weighted 0.876026687150048


In [ ]:
#запустить эту ячейку если метрика ансамбля лучше метрики одной модели (для того, чтобы использовать его в ансамбле разных моделей)

model_name = 'rf'
pred_dict[model_name] = rf_ens_pred
pred_val_dict[model_name] = rf_ensemble.predict(val_data_ens, mode='mode')
precision_dict[model_name] =  precision_score(val_data_ens['class'], pred_val_dict[model_name], average=None)

### Roberta

In [ ]:
import roberta

#инициализация входных данных модели
num_classes = len(all_data[target_name].unique())
epochs = 4
checkpoint_path = get_drive_path("saved models\\saved-model-roberta.hdf5")

roberta_processor = roberta.RobertaProcessor()

tokens_train, attn_mask_train, y_train = roberta_processor.get_data(train_data, all_data)
tokens_train, tokens_val, attn_mask_train, attn_mask_val, y_train, y_val = train_test_split(tokens_train, attn_mask_train, y_train, 
                                                                                            random_state=42, test_size=0.2)

In [ ]:
#инициализация и обучение модели
model = roberta.train_roberta_model(tokens_train, tokens_val, attn_mask_train, attn_mask_val, y_train, y_val, 
                                    num_classes, checkpoint_path, epochs)

model.load_weights(checkpoint_path)

In [ ]:
#вывод метрики на тестовой выборке, определение переменных для работы ансамбля
tokens_test, attn_mask_test, y_test = roberta_processor.get_data(test_data, train=False)

pred_roberta = roberta_processor.encoder.inverse_transform(conv.round_prediction(model.predict([tokens_test, attn_mask_test])))

print('f1 micro', f1_score(test_data['class'], pred_roberta, average='micro'))
print('f1 weighted', f1_score(test_data['class'], pred_roberta, average='weighted'))

model_name = 'roberta'
pred_dict[model_name] = pred_roberta
pred_val_dict[model_name] = roberta_processor.encoder.inverse_transform(conv.round_prediction(model.predict([tokens_val, attn_mask_val])))
precision_dict[model_name] =  precision_score(roberta_processor.encoder.inverse_transform(np.round(y_val)), pred_val_dict[model_name], average=None)

In [ ]:
#сохранение/загрузка предсказаний модели
roberta_pred_path = get_drive_path('saved models/roberta_pred.npz')
model_name = 'roberta'
try:
    np.savez(roberta_pred_path, pred_dict[model_name], pred_val_dict[model_name], precision_dict[model_name])
except KeyError:
    roberta_pred_dict = np.load(roberta_pred_path)
    pred_roberta, pred_roberta_val, precision_roberta = roberta_pred_dict['arr_0'], roberta_pred_dict['arr_1'], roberta_pred_dict['arr_2']

    pred_dict[model_name] = pred_roberta
    pred_val_dict[model_name] = pred_roberta_val
    precision_dict[model_name] = precision_roberta

#### Roberta Ensemble

In [ ]:
import scipy.stats as stats
import ensemble
importlib.reload(ensemble)

class RobertaEnsemble(ensemble.EnsembleTemplate):

    def __init__(self, num_classes, all_data, **kwargs):

        self.num_classes = num_classes
        self.all_data = all_data
        super().__init__(**kwargs)

    def train(self, **kwargs):

        print('train() method not supported, use train_and_predict() instead')

    def train_and_predict(self, data, pred_data, retrain=False, mode='mode'):

        self.pred_data = pred_data

        super().train(data, retrain)

        if mode == 'mode':
            predictions = np.hstack(self.predictions)
            return stats.mode(predictions, axis=1).mode
        
        else:
            print(f'"{mode}" mode not supported')

    def fit(self, data, val_data):

        if not getattr(self, 'roberta_processor', False):
            self.roberta_processor = RobertaProcessor()
            tokens_train, attn_mask_train, y_train = self.roberta_processor.get_data(train_data, self.all_data, train=True)
        
        else:
            tokens_train, attn_mask_train, y_train = self.roberta_processor.get_data(data, train=False)

        tokens_val, attn_mask_val, y_val = self.roberta_processor.get_data(val_data, train=False)

        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
        tf.config.experimental_connect_to_cluster(resolver)
        # This is the TPU initialization code that has to be at the beginning.
        tf.tpu.experimental.initialize_tpu_system(resolver)
        strategy = tf.distribute.TPUStrategy(resolver)

        early_stopping_monitor = EarlyStopping(
            monitor='val_loss',
            min_delta=0,
            patience=10,
            verbose=0,
            mode='auto',
            baseline=None,
            restore_best_weights=True
        )

        optimizer = tf.optimizers.Adam(learning_rate=1e-05, epsilon=1e-07)
        with strategy.scope():
            model = create_roberta_model(tokens_train, attn_mask_train, self.num_classes)
            model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

            model.fit(x=[tokens_train, attn_mask_train], y=y_train, batch_size=80, 
                        validation_data=([tokens_val, attn_mask_val], y_val), epochs=20, 
                        callbacks=[early_stopping_monitor])

        if not getattr(self, 'predictions', False):
            self.predictions = []
        
        self.predictions.append(self.model_predict(model, self.pred_data))

        return model
    
    def train_meta_model(self, data):

        pass

    def model_predict(self, model, data):

        tokens_test, attn_mask_test, y_test = self.roberta_processor.get_data(data, train=False)

        pred_roberta_binary = conv.round_prediction(model.predict([tokens_test, attn_mask_test]))
        prediction = self.roberta_processor.encoder.inverse_transform(pred_roberta_binary)

        return prediction.reshape(-1,1)

    def save_model(self, model, path):

        model.save_weights(path+'.h5')

    def load_model(self, path):
        
        model = create_roberta_model()
        model.load_weights(path+'.h5')

        return model

train_data_ens, val_data_ens = train_test_split(train_data, test_size=0.1, stratify=train_data['class'], random_state=42)

roberta_ensemble = RobertaEnsemble(n_splits=5,
                           model_folder_path=get_drive_path('saved models/roberta ensemble'), 
                           model_name='roberta ensemble',
                           num_classes=num_classes,
                           all_data=all_data)
roberta_ens_pred = roberta_ensemble.train_and_predict(train_data_ens, test_data, retrain=True)

INFO:tensorflow:Initializing the TPU system: grpc://10.119.175.26:8470
INFO:tensorflow:Clearing out eager caches
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:

In [ ]:
f1_score(test_data['class'], roberta_ens_pred, average='micro')

0.9270298047276464

In [ ]:
#запустить эту ячейку если метрика ансамбля лучше метрики одной модели
precision_roberta = precision_score(val_data_ens['class'], roberta_ensemble.predict(val_data_ens, mode='mode'), average=None)
pred_roberta = roberta_ens_pred
pred_roberta_binary = conv_processor.y_encoder.transform(roberta_ens_pred).toarray()

FailedPreconditionError: ignored

In [ ]:
#сохранение/загрузка предсказаний модели
roberta_pred_path = get_drive_path('saved models/roberta_ensemble_pred.npz')
model_name = 'roberta'
try:
    np.savez(roberta_pred_path, pred_dict[model_name], pred_val_dict[model_name], precision_dict[model_name])
except KeyError:
    roberta_pred_dict = np.load(roberta_pred_path)
    pred_roberta, pred_roberta_val, precision_roberta = roberta_pred_dict['arr_0'], roberta_pred_dict['arr_1'], roberta_pred_dict['arr_2']

    pred_dict[model_name] = pred_roberta
    pred_val_dict[model_name] = pred_roberta_val
    precision_dict[model_name] = precision_roberta

### Ensemble

In [ ]:
from itertools import combinations

def ensemble_predict(preds, precisions=None, vote=True):
    '''
    Ансамбль предсказаний

    Parameters:
    preds: iterable of np.array - контейнер предсказаний моделей (предсказания one-hot)
    precisions: iterable of np.array - контейнер всех precision моделей (если None, то не используется)
    vote: boolean - использовать ли голосование моделей (если False, берется предсказание модели, чем precision по своему предсказанному классу максимален)
    
    Returns:
    new_pred: np.array
    '''
    new_pred = np.zeros(preds[0].shape)

    if precisions is not None and not vote:
        for i,_ in enumerate(new_pred):
            preds_and_precisions_multiplied = [pred[i,:]*precision for pred,precision in zip(preds, precisions)]
            
            cls = np.argmax(np.vstack(preds_and_precisions_multiplied).max(axis=0))
            new_pred[i, cls] = 1

    elif precisions is not None and vote:
        for i,_ in enumerate(new_pred):
            pred_sum = np.sum([pred[i,:]*precision for pred,precision in zip(preds, precisions)], axis=0)
            cls = np.argmax(pred_sum)
            new_pred[i, cls] = 1

    elif precisions is None and vote:
        for i,_ in enumerate(new_pred):
            pred_sum = np.sum([pred[i,:] for pred in preds], axis=0)
            cls = np.argmax(pred_sum)
            new_pred[i, cls] = 1
    
    elif precisions is None and not vote:
        print('bro no')
        raise ValueError

    return new_pred

#### Выбор ответа при максимальном precision

In [ ]:
_, y_test, _ = conv_processor.get_data(test_data, train=False)

preds = np.array([conv_processor.y_encoder.transform(pred.reshape(-1,1)).toarray() for pred in pred_dict.values()])
precisions = np.array(list(precision_dict.values()))

n = len(preds)
idx = range(n) #индексы предсказаний моделей
names = np.array(list(pred_dict.keys()))
summary = []

#перебираем все возможные комбинации предсказаний
for r in range(1, n+1):
    for comb in combinations(idx, r):

        preds_comb = preds[list(comb)]
        precisions_comb = precisions[list(comb)]

        new_pred = ensemble_predict(preds_comb, precisions_comb, vote=False)

        summary.append([names[list(comb)], 
                        f1_score(y_test, new_pred, average='micro'),
                        f1_score(y_test, new_pred, average='weighted')])

summary = pd.DataFrame(summary, columns=['Комбинация','f1 micro','f1 weighted'])
summary.sort_values('f1 micro', ascending=False)

,Комбинация,f1 micro,f1 weighted
1,[bert],0.952724,0.949619
5,"[svm, bert]",0.951182,0.948050
15,"[svm, bert, rf]",0.949640,0.946463
9,"[bert, rf]",0.949126,0.944340
11,"[bert, roberta]",0.948613,0.945352
10,"[bert, conv]",0.948099,0.944833
16,"[svm, bert, conv]",0.948099,0.944902
17,"[svm, bert, roberta]",0.947585,0.944479
25,"[svm, bert, rf, conv]",0.946557,0.943271
26,"[svm, bert, rf, roberta]",0.946043,0.942779


#### Голосование

In [ ]:
summary = []

#перебираем все возможные комбинации предсказаний
for r in range(1, n+1):
    for comb in combinations(idx, r):

        preds_comb = preds[list(comb)]
        precisions_comb = precisions[list(comb)]

        new_pred = ensemble_predict(preds_comb, vote=True)

        summary.append([names[list(comb)], 
                        f1_score(y_test, new_pred, average='micro'),
                        f1_score(y_test, new_pred, average='weighted')])

summary = pd.DataFrame(summary, columns=['Комбинация','f1 micro','f1 weighted'])
summary.sort_values('f1 micro', ascending=False)

,Комбинация,f1 micro,f1 weighted
23,"[bert, conv, roberta]",0.955293,0.953475
30,"[svm, bert, rf, conv, roberta]",0.953237,0.951375
17,"[svm, bert, roberta]",0.953237,0.951223
1,[bert],0.952724,0.949619
27,"[svm, bert, conv, roberta]",0.951182,0.949449
29,"[bert, rf, conv, roberta]",0.949640,0.946867
22,"[bert, rf, roberta]",0.948613,0.945414
25,"[svm, bert, rf, conv]",0.948099,0.945509
16,"[svm, bert, conv]",0.948099,0.946403
21,"[bert, rf, conv]",0.947071,0.944218


#### Голосование precision

In [ ]:
summary = []

#перебираем все возможные комбинации предсказаний
for r in range(1, n+1):
    for comb in combinations(idx, r):

        preds_comb = preds[list(comb)]
        precisions_comb = precisions[list(comb)]

        new_pred = ensemble_predict(preds_comb, precisions_comb, vote=True)

        summary.append([names[list(comb)], 
                        f1_score(y_test, new_pred, average='micro'),
                        f1_score(y_test, new_pred, average='weighted')])

summary = pd.DataFrame(summary, columns=['Комбинация','f1 micro','f1 weighted'])
summary.sort_values('f1 micro', ascending=False)

,Комбинация,f1 micro,f1 weighted
17,"[svm, bert, roberta]",0.954779,0.951734
23,"[bert, conv, roberta]",0.953751,0.950906
26,"[svm, bert, rf, roberta]",0.953237,0.949398
1,[bert],0.952724,0.949619
29,"[bert, rf, conv, roberta]",0.952724,0.949106
30,"[svm, bert, rf, conv, roberta]",0.952210,0.948988
27,"[svm, bert, conv, roberta]",0.952210,0.949299
5,"[svm, bert]",0.951182,0.948050
22,"[bert, rf, roberta]",0.950154,0.946573
16,"[svm, bert, conv]",0.949640,0.947338


#### Stacking

In [ ]:
preds_val = np.hstack((pred_conv_val, pred_svm_val.reshape(-1,1), pred_bert_val.reshape(-1,1), pred_rf_val.reshape(-1,1)))
y_train_ens, _ = train_test_split(train_data[target_name], stratify=train_data[target_name], test_size=0.2, random_state=42)
preds_test = np.hstack((pred_conv, pred_svm.reshape(-1,1), pred_bert.reshape(-1,1), pred_rf.reshape(-1,1)))
y_test = test_data[target_name]

ValueError: ignored

In [ ]:
#one-hot аналог ячейки выше
pred_conv_val_binary = conv_processor.y_encoder.transform(pred_conv_val).toarray()
pred_svm_val_binary = conv_processor.y_encoder.transform(pred_svm_val.reshape(-1,1)).toarray()
pred_bert_val_binary = conv_processor.y_encoder.transform(pred_bert_val.reshape(-1,1)).toarray()
pred_rf_val_binary = conv_processor.y_encoder.transform(pred_rf_val.reshape(-1,1)).toarray()

preds_val = np.hstack((pred_conv_val_binary, pred_svm_val_binary, pred_bert_val_binary, pred_rf_val_binary))
_, y_val = train_test_split(train_data[target_name], stratify=train_data[target_name], test_size=0.2, random_state=42)
preds = np.hstack((pred_conv_binary, pred_svm_binary, pred_bert_binary, pred_rf_binary))
y_test = test_data[target_name]

NameError: ignored

#### Logistic Regression stacking

In [ ]:
from sklearn.linear_model import LogisticRegression

y_test = test_data[target_name]
preds_val_list = np.array([pred_svm_val_binary, pred_bert_val_binary, pred_rf_val_binary, pred_conv_val_binary])
preds_list = np.array([pred_svm_binary, pred_bert_binary, pred_rf_binary, pred_conv_binary])

n = len(preds_list)
idx = range(n) #какие предсказания используются
names = np.array(['svm','bert','rf','conv'])
summary = []

for r in range(1, n+1): #комбинации из i элементов
    for comb in combinations(idx, r):

        preds_val_list_comb = preds_val_list[list(comb)]
        preds_list_comb = preds_list[list(comb)]

        lr_model_ens = LogisticRegression(max_iter=1000, solver='lbfgs')

        preds_val_comb = np.hstack(preds_val_list_comb)
        preds_test_comb = np.hstack(preds_list_comb)

        lr_model_ens.fit(preds_val_comb, y_val)

        pred_ens = lr_model_ens.predict(preds_test_comb)

        summary.append([names[list(comb)], 
                        f1_score(y_test, pred_ens, average='micro'),
                        f1_score(y_test, pred_ens, average='weighted')])

summary = pd.DataFrame(summary, columns=['Комбинация','f1 micro','f1 weighted'])
summary.sort_values('f1 micro', ascending=False)

,Комбинация,f1 micro,f1 weighted
1,[bert],0.942241,0.937348
7,"[bert, rf]",0.942241,0.937348
8,"[bert, conv]",0.942241,0.937348
13,"[bert, rf, conv]",0.942241,0.937348
14,"[svm, bert, rf, conv]",0.940153,0.933889
10,"[svm, bert, rf]",0.939457,0.933038
4,"[svm, bert]",0.938761,0.932410
11,"[svm, bert, conv]",0.938065,0.931790
0,[svm],0.935978,0.932169
6,"[svm, conv]",0.935978,0.932142


#### SVM stacking

In [ ]:
#scaler_ensemble = MinMaxScaler()
#X_train_ens = scaler_ensemble.fit_transform(X_train_ens)
#X_test_ens = scaler_ensemble.transform(X_test_ens)

svm_model_ens = SVC()

params = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
          'C': [5, 1, 0.5, 0.1, 0.05],
          'max_iter': [-1]}

model_grid = GridSearchCV(svm_model_ens,
                          params,
                          cv=5, n_jobs=-1, verbose=1)

model_grid.fit(preds_val, y_val)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   43.0s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [5, 1, 0.5, 0.1, 0.05],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                         'max_iter': [-1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [ ]:
model_grid.best_params_

{'C': 1, 'kernel': 'sigmoid', 'max_iter': -1}

In [ ]:
svm_params = {'C': 1, 'kernel': 'linear', 'max_iter': -1}

In [ ]:
summary = []

for r in range(1, n+1): #комбинации из i элементов
    for comb in combinations(idx, r):

        preds_val_list_comb = preds_val_list[list(comb)]
        preds_list_comb = preds_list[list(comb)]

        svm_model_ens = SVC(**svm_params)

        preds_val_comb = np.hstack(preds_val_list_comb)
        preds_test_comb = np.hstack(preds_list_comb)

        svm_model_ens.fit(preds_val_comb, y_val)

        pred_ens = svm_model_ens.predict(preds_test_comb)

        summary.append([names[list(comb)], 
                        f1_score(y_test, pred_ens, average='micro'),
                        f1_score(y_test, pred_ens, average='weighted')])

summary = pd.DataFrame(summary, columns=['Комбинация','f1 micro','f1 weighted'])
summary.sort_values('f1 micro', ascending=False)

,Комбинация,f1 micro,f1 weighted
4,"[svm, bert]",0.943633,0.942096
11,"[svm, bert, conv]",0.942937,0.938737
14,"[svm, bert, rf, conv]",0.942937,0.938008
1,[bert],0.942241,0.937348
7,"[bert, rf]",0.942241,0.937348
8,"[bert, conv]",0.942241,0.937348
10,"[svm, bert, rf]",0.942241,0.938474
13,"[bert, rf, conv]",0.942241,0.937348
0,[svm],0.936674,0.933543
6,"[svm, conv]",0.935978,0.932795


### Rules + Ensemble

In [ ]:
!pip install razdel
!pip install pymorphy2
!sudo pip3 install rutimeparser
from nltk import download
download('stopwords')
download('punkt')
import rules

     |████████████████████████████████| 61kB 5.6MB/s 
     |████████████████████████████████| 8.2MB 9.7MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#предсказания и precision моделей из лучшего ансамбля
comb = ['bert']
preds_comb = np.array([conv_processor.y_encoder.transform(pred_dict[model_name].reshape(-1,1)).toarray() for model_name in comb])
precisions_comb = np.array([precision_dict[model_name] for model_name in comb])

code_names_path, sentences_features_path = get_drive_path('data/Классы_кодировка.csv'), get_drive_path('data/Фичи_17-12-2020.xlsx')
rule_predictor = rules.RulePredictor(all_data, code_names_path, sentences_features_path)

remove_bad_rules = True #убрать правила с плохой производительностью (для этого запустить текущую и след ячейку, а потом еще раз текущую)
try:
    if not remove_bad_rules:
        raise NameError
    for i in bad_rules[::-1]:
        rule_predictor.cond_class.pop(i)
except NameError:
    pass

rule_preds_list = rule_predictor.classify_with_rules(test_data, use_key_sentences=False) #предсказание на основе правил и ключевых слов, если предсказания нет, то будет возвращено None
ensemble_rule_pred = conv_processor.y_encoder.inverse_transform(ensemble_predict(preds_comb, precisions_comb, vote=True))
for i,prediction in enumerate(rule_preds_list):
    if prediction is not None:
        ensemble_rule_pred[i] = prediction

print('f1 micro', f1_score(test_data[target_name], ensemble_rule_pred, average='micro'))
print('f1 weighted', f1_score(test_data[target_name], ensemble_rule_pred, average='weighted'))

f1 micro 0.9552929085303186
f1 weighted 0.9534199286692941


In [ ]:
summary = pd.DataFrame(rule_predictor.conds_summary.values(), index=rule_predictor.conds_summary.keys(), columns=['Правильных предсказаний', 'Всего предсказаний'])
summary['Точность'] = summary['Правильных предсказаний'] / summary['Всего предсказаний']
bad_rules = summary.index[summary['Точность'] < 0.9]

### ArangoDB

#### Создание тестовой БД

In [ ]:
edges = all_data[['payer_bank_bic_cd', 'payee_bank_bic_cd', 'sum', 'purp', 'name_to', 'name_from']]
edges = edges.rename({'payer_bank_bic_cd':'_from', 'payee_bank_bic_cd':'_to'}, axis=1)
edges.to_csv('GPB_edges.tsv', index=False, sep='\t')

nodes = pd.concat((all_data['payer_bank_bic_cd'], all_data['payee_bank_bic_cd'])).unique()
nodes = pd.DataFrame(nodes, columns=['_key'])
nodes.to_csv('GPB_nodes.csv', index=False)

In [ ]:
def best_model_predict(data):
    return bert_ensemble.predict(data, mode='mode')

!pip install pyArango --user
import pyArango.connection as connection
conn = connection.Connection(username="root", password="progasila")
db = conn.createDatabase(name="named _system")




ModuleNotFoundError: ignored